In [36]:
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score

import numpy as np
import tensorflow as tf

import keras
import os
import sys

sys.path.append('..')
import data
import model
import preprocessing
import utils

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:
params = utils.yaml_to_dict(os.path.join('..','config.yml'))
params['data_dir'] = os.path.join('..',params['data_dir'])
params['model_dir'] = os.path.join('..',params['model_dir'])

## Primer paso: preprocesar los datos 
Hacerlo solo una vez

In [30]:
#preprocessing.download_data(params)
#preprocessing.extract_data(params)
#preprocessing.generate_spectogram_images(params)
#preprocessing.make_id_label_map(params)
preprocessing.split_data(params)

## Segund Paso: instanciar el modelo

In [31]:
inputs = tf.keras.layers.Input(shape=(110,480, 3))
net = model.ModelArchitecture(num_classes=params['num_classes'])
x = net(inputs, training=False)
net.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 55, 240, 3)        444       
_________________________________________________________________
batch_normalization_2 (Batch (None, 55, 240, 3)        12        
_________________________________________________________________
activation_3 (Activation)    (None, 55, 240, 3)        0         
_________________________________________________________________
global_average_pooling2d_1 ( (None, 3)                 0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               512       
_________________________________________________________________
batch_normalization_3 (Batch (None, 128)               512       
_________________________________________________________________
activation_4 (Activation)    (None, 128)               0         
__________

## Tercer paso: crear el generador

In [32]:
print('Data in train')
train_generator = data.make_datagenerator(params, mode='training')
print('Data in validation')
val_generator = data.make_datagenerator(params,mode='validation')

Data in train
Found 2960 images.
Data in validation
Found 30 images.


## Cuarto paso: correr el modelo

In [33]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    os.path.join(params['model_dir'], 'tf_ckpt'), 
    save_weights_only=True, 
    verbose=1,
    period=5)

tb_callback = tf.keras.callbacks.TensorBoard(
    os.path.join(params['model_dir'], 'logs'))

optimizer = tf.keras.optimizers.Adam(params['learning_rate'])

steps_per_epoch = train_generator.n // params['batch_size']
validation_steps = val_generator.n // params['batch_size']

net.compile(optimizer=optimizer, loss=params['loss'], metrics=['sparse_categorical_accuracy'])
net.fit_generator(
    train_generator, 
    steps_per_epoch=steps_per_epoch, 
    epochs=params['num_epochs'],
    workers=4,
    validation_data=val_generator, 
    validation_steps=validation_steps,
    callbacks=[cp_callback,tb_callback])

Epoch 1/10
370/370 [==============================] - 22s 59ms/step - loss: 2.3011 - sparse_categorical_accuracy: 0.1118 - val_loss: 2.2841 - val_sparse_categorical_accuracy: 0.1667
Epoch 2/10
370/370 [==============================] - 23s 63ms/step - loss: 2.2990 - sparse_categorical_accuracy: 0.1115 - val_loss: 2.2838 - val_sparse_categorical_accuracy: 0.1667
Epoch 3/10
370/370 [==============================] - 21s 58ms/step - loss: 2.2980 - sparse_categorical_accuracy: 0.1166 - val_loss: 2.2767 - val_sparse_categorical_accuracy: 0.1667
Epoch 4/10
370/370 [==============================] - 18s 48ms/step - loss: 2.2972 - sparse_categorical_accuracy: 0.1135 - val_loss: 2.2792 - val_sparse_categorical_accuracy: 0.1667
Epoch 5/10
368/370 [============================>.] - ETA: 0s - loss: 2.2945 - sparse_categorical_accuracy: 0.1165
Epoch 00005: saving model to ..\checkpoints\tf_ckpt

Consider using a TensorFlow optimizer from `tf.train`.
370/370 [==============================] - 17s 47

In [34]:
#fpr, tpr, thresholds = roc_curve(y, probs)

In [35]:
# Quintero paso: evaluar el modelo

In [37]:
params['batch_size'] = val_generator.n
params['shuffle'] = False
val_generator = data.make_datagenerator(params,mode='validation')

Found 30 images.


In [38]:
Y_pred = net.predict_generator(val_generator, steps=1)
y_pred = np.argmax(Y_pred, axis=1)

In [39]:
print('Confusion Matrix')
print(confusion_matrix(val_generator.data, y_pred))
target_names = ["arranca", "acelera", "pita", "izquierda","frena", "detente", "retrocede", "gira", "avanza", "derecha"]
print(classification_report(val_generator.data, y_pred, target_names=target_names))

Confusion Matrix
[[0 1 0 0 0 0 0 0 1 0]
 [0 1 2 1 0 0 0 0 0 0]
 [0 3 0 1 0 0 0 0 0 0]
 [0 1 2 1 0 0 0 0 0 0]
 [0 1 0 1 0 0 0 0 1 0]
 [0 3 0 0 0 0 0 0 0 0]
 [0 0 1 2 0 0 0 0 0 0]
 [0 1 1 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 2 0]
 [0 1 0 0 0 0 0 0 1 0]]
              precision    recall  f1-score   support

     arranca       0.00      0.00      0.00         2
     acelera       0.08      0.25      0.12         4
        pita       0.00      0.00      0.00         4
   izquierda       0.17      0.25      0.20         4
       frena       0.00      0.00      0.00         3
     detente       0.00      0.00      0.00         3
   retrocede       0.00      0.00      0.00         3
        gira       0.00      0.00      0.00         2
      avanza       0.40      0.67      0.50         3
     derecha       0.00      0.00      0.00         2

   micro avg       0.13      0.13      0.13        30
   macro avg       0.06      0.12      0.08        30
weighted avg       0.07      0.13      0.09     